## up-ac Basic Example

This python notebook shows an examplary usage of the up-ac module.

Project page: [![Open GitHub](https://img.shields.io/badge/see-Github-579aca?logo=github)](https://github.com/DimitriWeiss/up-ac)

In particulat, this notebook will go through an example of Algorithm Configuration for ENHSP, an Expressic Numeric Heuristic Search Planner. For this we use SMAC, a Sequential Model-Based Optimization for
General Algorithm Configuration.

### Dependencies

We are updating Java because the default version on the notebook is not up to date enough for the Java version of enhsp. Then, we install the up_ac library from GitHub. We use the `--pre` flag to install the latest build dependencies for our `enhsp` planner.

In [ ]:
!apt install openjdk-17-jdk openjdk-17-jre
!pip install git+https://github.com/DimitriWeiss/up-ac.git
!pip install --pre unified-planning[enhsp]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-17-jdk is already the newest version (17.0.8.1+1~us1-0ubuntu1~22.04).
openjdk-17-jre is already the newest version (17.0.8.1+1~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
  Cloning https://github.com/DimitriWeiss/up-ac.git to /tmp/pip-req-build-n1lp2sek
  Running command git clone --filter=blob:none --quiet https://github.com/DimitriWeiss/up-ac.git /tmp/pip-req-build-n1lp2sek
  Resolved https://github.com/DimitriWeiss/up-ac.git to commit ae7a6f9d2aca1e6321fb80757cd698be2f9bab4c
  Preparing metadata (setup.py) ... done


Now, we are ready to use the up-ac module.

### Imports

In [ ]:
import os
import up_ac
import unified_planning as up
import multiprocessing as mp
from unified_planning.io import PDDLReader

As we use SMAC to optimize our planner, we import the classes `SmacInterface` and `SmacConfigurator` from the up-ac module.

In [ ]:
from up_ac.Smac_interface import SmacInterface
from up_ac.Smac_configurator import SmacConfigurator

### Planner

We want to use `enhsp` to solve our problem instances, optimizing for both `runtime` and `quality` in seperate optimizations.

In [ ]:
engine = ['enhsp']
metrics = ['runtime', 'quality']

### Accessing Files
For testing purposes, there are multiple files included within the up-ac module. Hence we set `path` to access the imported modules files.
To enable the module access different files within itself, we change the current working directory to the installed python packages.

In [ ]:
path = '/usr/local/lib/python3.10/dist-packages/up_ac'

os.chdir("/usr/local/lib/python3.10/dist-packages")

### Problem representation
In this example, we will generate plans for the following problem instances. The problem instance files are located within the up-ac module.

In [ ]:
instances = [f'{path}/test_problems/depot/problem.pddl',
             f'{path}/test_problems/counters/problem.pddl',
             f'{path}/test_problems/citycar/problem.pddl',
             f'{path}/test_problems/sailing/problem.pddl',
             f'{path}/test_problems/safe_road/problem.pddl']

###Initialization
We initialize the generic Algorithm Configuration interface by calling it's respective class `SmacInterface` and reading in it's engine pieces. These are handed inside the up-ac package.

In [ ]:
sgaci = SmacInterface()
sgaci.read_engine_pcs(engine, f'{path}/engine_pcs')

We initialize an empty dictionary `instance_features` and compute features for every testing instance respectively.

In [ ]:
instance_features = {}
for instance in instances:
    instance_features[instance] \
        = sgaci.compute_instance_features(
            instance.rsplit('/', 1)[0] + '/domain.pddl',
            instance)

[WARNING][reader.py:78] Domain names as declared in domain and instance files do not coincide: "depot" vs " Depot"


We introduce `SAC` as our Smac Algorithm Configurator and save the `instances_features` from above in it. We then set both the training instances and testing instances to the included problem instances.
Before we can use this Configurator, we need to set a scenario. This declares how we want our Configurator to behave.

In [ ]:
SAC = SmacConfigurator()
SAC.get_instance_features(instance_features)
SAC.set_training_instance_set(instances)
SAC.set_test_instance_set(instances)

SAC.set_scenario(engine = engine[0],
                 param_space = sgaci.engine_param_spaces[engine[0]],
                 gaci = sgaci,
                 configuration_time=30,
                 n_trials=50,
                 min_budget=2,
                 max_budget=5,
                 crash_cost=0,
                 planner_timelimit=5,
                 n_workers=1,
                 instance_features = SAC.instance_features)


Setting instance features.


Setting training instance set.


Setting testing instance set.


SMAC scenario is set.



The `default_config` is a set of parameters, but it might not perform very well.

In [ ]:
default_config = sgaci.engine_param_spaces[engine[0]].get_default_configuration()

Since SMACs parallelization uses dask, we are required to pickle the feedback function. This is the reason for the necessity to pass the PDDLReader object. This is handled in the automated algorithm configurator internally.

In [ ]:
reader = PDDLReader()

### Simple Testing
Let's start with a simple test, by creating a feedback function and letting it evaluate the 'default_config' on our first testing instance.
To do so, we first need to set the scenario for our Configurator. As this function depends on a `metric`, we do this for both of the metrics we evaluate by.

In [ ]:
SAC_fb_func_quality = SAC.get_feedback_function(gaci = sgaci,
                                                engine = engine[0],
                                                metric = "quality",
                                                mode = 'OneshotPlanner')

SAC_fb_func_quality(default_config, instances[0], seed = 42, reader=reader)

{'heuristic': 'hadd', 'search_algorithm': 'gbfs'}
RESULT status: SOLVED_SATISFICING
engine: SAT-enhsp
plan: SequentialPlan:
    lift(hoist1, crate0, pallet1, distributor0)
    lift(hoist0, crate1, pallet0, depot0)
    load(hoist0, crate1, truck1, depot0)
    drive(truck1, depot0, distributor0)
    load(hoist1, crate0, truck1, distributor0)
    unload(hoist1, crate1, truck1, distributor0)
    drive(truck1, distributor0, distributor1)
    unload(hoist2, crate0, truck1, distributor1)
    drop(hoist1, crate1, pallet1, distributor0)
    drop(hoist2, crate0, pallet2, distributor1)
[LogMessage(level=<LogLevel.INFO: 2>, message='Domain parsed\nProblem parsed\nGrounding..\nGrounding Time: 70\nAibr Preprocessing\n|F|:40\n|X|:0\n|A|:90\n|P|:0\n|E|:0\nH1 Setup Time (msec): 19\nSetting horizon to:NaN\nRunning Greedy Best First Search\nh(n = s_0)=11.0\n g(n)= 1.0 h(n)=10.0\n g(n)= 2.0 h(n)=9.0\n g(n)= 3.0 h(n)=8.0\n g(n)= 4.0 h(n)=7.0\n g(n)= 5.0 h(n)=5.0\n g(n)= 6.0 h(n)=4.0\n g(n)= 7.0 h(n)=3.0\n 

-10.0

In [ ]:
SAC_fb_func_runtime = SAC.get_feedback_function(gaci = sgaci,
                                                engine = engine[0],
                                                metric = "runtime",
                                                mode = 'OneshotPlanner')

SAC_fb_func_runtime(default_config, instances[0], seed = 42, reader=reader)

{'heuristic': 'hadd', 'search_algorithm': 'gbfs'}
RESULT status: SOLVED_SATISFICING
engine: SAT-enhsp
plan: SequentialPlan:
    lift(hoist1, crate0, pallet1, distributor0)
    lift(hoist0, crate1, pallet0, depot0)
    load(hoist0, crate1, truck1, depot0)
    drive(truck1, depot0, distributor0)
    load(hoist1, crate0, truck1, distributor0)
    unload(hoist1, crate1, truck1, distributor0)
    drive(truck1, distributor0, distributor1)
    unload(hoist2, crate0, truck1, distributor1)
    drop(hoist1, crate1, pallet1, distributor0)
    drop(hoist2, crate0, pallet2, distributor1)
[LogMessage(level=<LogLevel.INFO: 2>, message='Domain parsed\nProblem parsed\nGrounding..\nGrounding Time: 51\nAibr Preprocessing\n|F|:40\n|X|:0\n|A|:90\n|P|:0\n|E|:0\nH1 Setup Time (msec): 9\nSetting horizon to:NaN\nRunning Greedy Best First Search\nh(n = s_0)=11.0\n g(n)= 1.0 h(n)=10.0\n g(n)= 2.0 h(n)=9.0\n g(n)= 3.0 h(n)=8.0\n g(n)= 4.0 h(n)=7.0\n g(n)= 5.0 h(n)=5.0\n g(n)= 6.0 h(n)=4.0\n g(n)= 7.0 h(n)=3.0\n g

3.66

Before starting, we supress a repeating call of engine credits.

In [ ]:
up.shortcuts.get_environment().credits_stream = None

### Running the Smac Algorithm Configurator

We initiate the SMAC Algorithm Configurator as we did above. We then run the optimization of our setup and save the best configuration as `incumbent`. After evaluating the performance of our optimized configuration we save it as a seperate file.

In [ ]:
for metric in metrics:

    SAC.set_scenario(engine[0],
                     sgaci.engine_param_spaces[engine[0]],
                     sgaci,
                     configuration_time=30,
                     n_trials=50,
                     min_budget=2,
                     max_budget=5,
                     crash_cost=0,
                     planner_timelimit=5,
                     n_workers=1,
                     instance_features=SAC.instance_features)


    SAC_fb_func = SAC.get_feedback_function(sgaci, engine[0],
                                            metric, 'OneshotPlanner')


    # run algorithm configuration
    incumbent, _ = SAC.optimize(feedback_function=SAC_fb_func)

    # check configurations performance
    perf = SAC.evaluate(metric, engine[0], 'OneshotPlanner',
                            incumbent, sgaci, planner_timelimit=5)

    # save best configuration found
    SAC.save_config('.', incumbent, sgaci, engine[0])


SMAC scenario is set.


Starting Parameter optimization

[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 20 to 12 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using -2 initial design configurations and 3 additional configurations.
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
{'heuristic': 'hadd', 'search_algorithm': 'gbfs'}
RESULT status: SOLVED_SATISFICING
engine: SAT-enhsp
plan: SequentialPlan:
[LogMessage(level=<LogLevel.INFO: 2>, message='Domain parsed\nProblem parsed\nGrounding..\nGrounding Time: 33\nAibr Preprocessing\n|F|:9\n|X|:0\n|A|:10\n|P|:0\n|E|:0\nH1 Setup Time (msec): 16\nSetting horizon to:NaN\nRunning Greedy Best First Search\nh(n = s_0)=0.0\nProblem Solved\n\nFound Plan:\n\nPlan-Length:0\nMetric (Search):0.0\nPlanning Time (msec): 433\nHeuristic Time (msec): 0\nSearch Time (msec): 8\nExpanded Nodes:1\nStates Evaluated:0\nFixed constraint violations during search (zero-crossi